# PYACL推理开发:NLP：TextCNN模型离线推理指导

## 实验简介

textCNN 模型源于 2014 年自然语言处理（NLP）领域对文本分类的需求。当时，传统的文本分类方法如支持向量机（SVM）和朴素贝叶斯主要依赖词袋模型或 n-gram 特征，但这些方法难以捕捉词语的语义和上下文。受计算机视觉中 CNN 成功的启发，研究者开始探索将 CNN 应用于文本数据，将句子视为一维“图像”，通过卷积操作捕获局部模式。

TextCNN是一种用于文本分类和文本特征提取的深度学习模型，它结合了卷积神经网络（CNN）的思想和自然语言处理（NLP）的需求。通过本实验，你将深入了解TextCNN模型的特点和优势、推理环境准备、代码下载、数据集准备和模型推理的全流程。掌握这些知识将帮助你在自然语言处理项目中更高效地应用TextCNN模型。

![TextCNN模型](https://model-community-picture.obs.cn-north-4.myhuaweicloud.com/ascend-zone/course/images_OVhEPc2kY53QSIkPJQN-b.png)

##  实验目的

- 了解TextCNN模型相关的细节知识及相关环境；
- 掌握TextCNN模型的推理方法。

##  模型简介
### 模型的特点和优势
TextCNN（Text Convolutional Neural Network）是一种结合了卷积神经网络（CNN）和自然语言处理（NLP）的深度学习模型，主要用于文本分类和文本特征提取。其具有以下优势：
1. 局部特征提取：TextCNN通过卷积层中多个不同size大小的kernel来实现更好捕捉文本中的局部结构信息，如短语或片语。
2. 参数共享：卷积层中的滤波器在输入序列上滑动时共享权重，这有助于模型在处理变长输入时减少参数数量。
3. 平移不变性：通过使用不同的滤波器，TextCNN可以捕捉到文本中的不同级别的特征，这使得模型具有平移不变性。
但也同时具备以下限制：
1. 局部特征依赖：TextCNN主要依赖局部特征，可能无法捕捉到长距离依赖关系。
2. 固定长度输入：TextCNN通常需要将变长输入转换为固定长度，这可能会导致一些信息丢失。

模型有多种变体：
- **CNN-rand**：词向量随机初始化并在训练中更新。
- **CNN-static**：使用预训练词向量，保持静态，不更新。
- **CNN-non-static**：使用预训练词向量，并在训练中微调。
- **CNN-multichannel**：同时使用静态和非静态词向量，结合两者的优势。

> 更多模型信息可参考[Convolutional Neural Networks for Sentence Classification](https://aclanthology.org/D14-1181.pdf)


### 模型独特的结构设计
1. 文本表示：TextCNN首先将文本数据表示为一个二维矩阵，其中每一行对应一个词，列数为词向量的维度。并且词向量通常通过预训练的词嵌入获得。
2. 卷积层：通过滤波器在词向量矩阵上进行卷积，从而捕捉局部的文本特征。每个卷积层生成一个特征图，每个特征图表示一个不同的文本模式。通过调整滤波器的大小和数量，TextCNN可以捕捉到不同级别的特征。
3. 多通道结合：TextCNN使用多通道结构，并且每个通道使用不同的滤波器从而捕捉不同层次的特征。
## 实验步骤

### 1 TextCNN模型实践

#### 1.1 获取模型源码：

In [1]:
#下载textcnn模型源码

!wget https://model-community-picture.obs.cn-north-4.myhuaweicloud.com/ascend-zone/notebook_codes/692e0de40a1611f0b284fa163edcddae/textcnn.zip

--2025-06-13 18:48:04--  https://model-community-picture.obs.cn-north-4.myhuaweicloud.com/ascend-zone/notebook_codes/692e0de40a1611f0b284fa163edcddae/textcnn.zip
Resolving model-community-picture.obs.cn-north-4.myhuaweicloud.com (model-community-picture.obs.cn-north-4.myhuaweicloud.com)... 121.36.121.227, 121.36.121.226
Connecting to model-community-picture.obs.cn-north-4.myhuaweicloud.com (model-community-picture.obs.cn-north-4.myhuaweicloud.com)|121.36.121.227|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14083 (14K) [application/zip]
Saving to: 'textcnn.zip'

textcnn.zip         100%[===================>]  13.75K  --.-KB/s    in 0.01s   

2025-06-13 18:48:04 (1.33 MB/s) - 'textcnn.zip' saved [14083/14083]



In [2]:
#解压textcnn模型源码
!unzip textcnn.zip

Archive:  textcnn.zip
   creating: textcnn/
  inflating: textcnn/requirements.txt  
  inflating: textcnn/License         
  inflating: textcnn/TextCNN_postprocess.py  
  inflating: textcnn/TextCNN_preprocess.py  
  inflating: textcnn/README.md       
  inflating: textcnn/modelzoo_level.txt  
  inflating: textcnn/gen_dataset_info.py  
  inflating: textcnn/fix_onnx.py     
  inflating: textcnn/TextCNN_pth2onnx.py  


将工作目录切换至textcnn模型文件夹下：

In [3]:
%cd ./textcnn

/opt/huawei/edu-apaas/src/init/textcnn


#####  1.1.1 获取昇腾textcnn源码：

In [4]:
!wget https://model-community-picture.obs.cn-north-4.myhuaweicloud.com/ascend-zone/notebook_codes/692e0de40a1611f0b284fa163edcddae/ascend-textcnn.zip

--2025-06-13 18:48:11--  https://model-community-picture.obs.cn-north-4.myhuaweicloud.com/ascend-zone/notebook_codes/692e0de40a1611f0b284fa163edcddae/ascend-textcnn.zip
Resolving model-community-picture.obs.cn-north-4.myhuaweicloud.com (model-community-picture.obs.cn-north-4.myhuaweicloud.com)... 121.36.121.227, 121.36.121.226
Connecting to model-community-picture.obs.cn-north-4.myhuaweicloud.com (model-community-picture.obs.cn-north-4.myhuaweicloud.com)|121.36.121.227|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 181809210 (173M) [application/zip]
Saving to: 'ascend-textcnn.zip'

ascend-textcnn.zip  100%[===================>] 173.39M  38.4MB/s    in 4.7s    

2025-06-13 18:48:16 (37.3 MB/s) - 'ascend-textcnn.zip' saved [181809210/181809210]



In [5]:
#解压ascend-textcnn源码
!unzip ascend-textcnn.zip

Archive:  ascend-textcnn.zip
   creating: ascend-textcnn/
  inflating: ascend-textcnn/preprocess_to_bin.py  
   creating: ascend-textcnn/.git/
 extracting: ascend-textcnn/.git/ORIG_HEAD  
  inflating: ascend-textcnn/.git/description  
   creating: ascend-textcnn/.git/hooks/
  inflating: ascend-textcnn/.git/hooks/pre-rebase.sample  
  inflating: ascend-textcnn/.git/hooks/push-to-checkout.sample  
  inflating: ascend-textcnn/.git/hooks/pre-merge-commit.sample  
  inflating: ascend-textcnn/.git/hooks/prepare-commit-msg.sample  
  inflating: ascend-textcnn/.git/hooks/pre-push.sample  
  inflating: ascend-textcnn/.git/hooks/fsmonitor-watchman.sample  
  inflating: ascend-textcnn/.git/hooks/pre-applypatch.sample  
  inflating: ascend-textcnn/.git/hooks/applypatch-msg.sample  
  inflating: ascend-textcnn/.git/hooks/post-update.sample  
  inflating: ascend-textcnn/.git/hooks/commit-msg.sample  
  inflating: ascend-textcnn/.git/hooks/pre-commit.sample  
  inflating: ascend-textcnn/.git/hooks/up

#### 1.2 环境准备
##### 1.2.1 获取msadvisor并进行master分支检查

msadvisor 是华为昇腾 AI 处理器提供的性能调优专家系统工具，旨在帮助开发者识别算子或模型的性能瓶颈，并提供合理的性能调优方案。通过 msadvisor，开发者可以准确识别算子或模型的性能瓶颈，输出合理的性能调优方案，甚至直接完成性能调优操作，提升 AI 应用的性能表现。

auto-optimizer（自动调优工具）使能ONNX模型在昇腾芯片的优化，并提供基于ONNX的改图功能。

auto-optimizer主要通过graph_optimizer、knowledge、graph_refactor和inference_component接口提供专家系统工具的自动调优能力。

In [6]:
#下载并且解压msadviso源码
!wget https://model-community-picture.obs.cn-north-4.myhuaweicloud.com/ascend-zone/notebook_codes/692e0de40a1611f0b284fa163edcddae/msadvisor.zip

!unzip msadvisor.zip

--2025-06-13 18:48:20--  https://model-community-picture.obs.cn-north-4.myhuaweicloud.com/ascend-zone/notebook_codes/692e0de40a1611f0b284fa163edcddae/msadvisor.zip
Resolving model-community-picture.obs.cn-north-4.myhuaweicloud.com (model-community-picture.obs.cn-north-4.myhuaweicloud.com)... 121.36.121.227, 121.36.121.226
Connecting to model-community-picture.obs.cn-north-4.myhuaweicloud.com (model-community-picture.obs.cn-north-4.myhuaweicloud.com)|121.36.121.227|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15014568 (14M) [application/zip]
Saving to: 'msadvisor.zip'

msadvisor.zip       100%[===================>]  14.32M  23.2MB/s    in 0.6s    

2025-06-13 18:48:21 (23.2 MB/s) - 'msadvisor.zip' saved [15014568/15014568]

Archive:  msadvisor.zip
   creating: msadvisor/
   creating: msadvisor/training/
  inflating: msadvisor/training/__main__.py  
 extracting: msadvisor/training/requirements.txt  
  inflating: msadvisor/training/README.md  
  inflating: msa


由于有改图部分，因此需要安装auto-optimizer

In [7]:
%cd msadvisor/auto-optimizer
%pip install --upgrade pip
%pip install wheel
%pip install ./
%cd ../..

/opt/huawei/edu-apaas/src/init/textcnn/msadvisor/auto-optimizer
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://mirrors.huaweicloud.com/repository/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.3 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://mirrors.huaweicloud.com/repository/pypi/simple
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writea

##### 1.2.2 安装模型requirement依赖

In [8]:
%pip install numpy==1.24.4
%pip install Pillow
%pip install torch==2.1.0
%pip install torchvision==0.16.0
%pip install onnx
%pip install attrs  #为后续atc工具转换提供依赖
%pip install packaging==24.2

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://mirrors.huaweicloud.com/repository/pypi/simple
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://mirrors.huaweicloud.com/repository/pypi/simple
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://mirrors.huaweicloud.com/repository/pypi/simple
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://mirrors.huaweicloud.com/repository/pypi/simple
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://mirrors.huaweiclou

##### 1.2.3 推理工具安装
使用ais-bench工具进行推理。

请访问[ais_bench推理工具代码仓](https://gitee.com/ascend/tools/tree/master/ais-bench_workload/tool/ais_bench)，根据readme文档进行工具安装。

In [9]:
# 查看系统架构
!uname -a

Linux notebook-4461b0d0edb84acca99965fb130f8291-7949f84bf5-9xh7t 4.19.90-vhulk2211.3.0.h1543.eulerosv2r10.aarch64 #1 SMP Tue Jun 6 07:58:07 UTC 2023 aarch64 aarch64 aarch64 GNU/Linux


In [10]:
#下载aclruntime以及ais_bench whl包 以本实验系统架构aarch64为例
# aarch64架构aclruntime安装包
!wget https://model-community-picture.obs.cn-north-4.myhuaweicloud.com/ascend-zone/notebook_codes/692e0de40a1611f0b284fa163edcddae/aclruntime-0.0.2-cp38-cp38-linux_aarch64.whl
# ais_bench安装包
!wget https://model-community-picture.obs.cn-north-4.myhuaweicloud.com/ascend-zone/notebook_codes/692e0de40a1611f0b284fa163edcddae/ais_bench-0.0.2-py3-none-any.whl

--2025-06-13 18:49:06--  https://model-community-picture.obs.cn-north-4.myhuaweicloud.com/ascend-zone/notebook_codes/692e0de40a1611f0b284fa163edcddae/aclruntime-0.0.2-cp38-cp38-linux_aarch64.whl
Resolving model-community-picture.obs.cn-north-4.myhuaweicloud.com (model-community-picture.obs.cn-north-4.myhuaweicloud.com)... 121.36.121.227, 121.36.121.226
Connecting to model-community-picture.obs.cn-north-4.myhuaweicloud.com (model-community-picture.obs.cn-north-4.myhuaweicloud.com)|121.36.121.227|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 469829 (459K) [binary/octet-stream]
Saving to: 'aclruntime-0.0.2-cp38-cp38-linux_aarch64.whl'

aclruntime-0.0.2-cp 100%[===================>] 458.82K  2.69MB/s    in 0.2s    

2025-06-13 18:49:07 (2.69 MB/s) - 'aclruntime-0.0.2-cp38-cp38-linux_aarch64.whl' saved [469829/469829]

--2025-06-13 18:49:07--  https://model-community-picture.obs.cn-north-4.myhuaweicloud.com/ascend-zone/notebook_codes/692e0de40a1611f0b284fa163edcd

In [11]:
# 安装aclruntime
%pip install ./aclruntime-0.0.2-cp38-cp38-linux_aarch64.whl
# 安装ais_bench推理程序
%pip install ./ais_bench-0.0.2-py3-none-any.whl

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://mirrors.huaweicloud.com/repository/pypi/simple
Processing ./aclruntime-0.0.2-cp38-cp38-linux_aarch64.whl
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://mirrors.huaweicloud.com/repository/pypi/simple
Processing ./ais_bench-0.0.2-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


#### 1.3 获取数据集
##### 1.3.1 使用昇腾textcnn数据集
本模型支持前文下载仓库ascend-textcnn内的Chinese-Text-Classification-Pytorch/THUCNews

##### 1.3.2数据预处理

In [12]:
%cd ascend-textcnn
!mv ../TextCNN_preprocess.py ./
!mv ../TextCNN_pth2onnx.py ./
!mv ../TextCNN_postprocess.py ./
!mv ../gen_dataset_info.py ./
import sys
!{sys.executable} TextCNN_preprocess.py --save_folder bin
!{sys.executable} gen_dataset_info.py bin info

/opt/huawei/edu-apaas/src/init/textcnn/ascend-textcnn
Vocab size: 4762
bin file save path:  /opt/huawei/edu-apaas/src/init/textcnn/ascend-textcnn/bin


参数说明：

- save_folder：保存二进制数据集的路径。

- bin：读取二进制数据集的路径。

- info：生成的信息文件

#### 1.4 模型推理
##### 1.4.1 模型转换
使用PyTorch将模型权重文件.pth转换为.onnx文件，再使用ATC工具将.onnx文件转为离线推理模型文件.om文件。
1. 获取权重文件。

In [13]:
!wget https://model-community-picture.obs.cn-north-4.myhuaweicloud.com/ascend-zone/notebook_models/7a0fc4360a1611f0b284fa163edcddae/TextCNN_9045_seed460473.pth

--2025-06-13 18:49:19--  https://model-community-picture.obs.cn-north-4.myhuaweicloud.com/ascend-zone/notebook_models/7a0fc4360a1611f0b284fa163edcddae/TextCNN_9045_seed460473.pth
Resolving model-community-picture.obs.cn-north-4.myhuaweicloud.com (model-community-picture.obs.cn-north-4.myhuaweicloud.com)... 121.36.121.227, 121.36.121.226
Connecting to model-community-picture.obs.cn-north-4.myhuaweicloud.com (model-community-picture.obs.cn-north-4.myhuaweicloud.com)|121.36.121.227|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8649678 (8.2M) [binary/octet-stream]
Saving to: 'TextCNN_9045_seed460473.pth'

TextCNN_9045_seed46 100%[===================>]   8.25M  20.3MB/s    in 0.4s    

2025-06-13 18:49:20 (20.3 MB/s) - 'TextCNN_9045_seed460473.pth' saved [8649678/8649678]



2. 导出onnx文件
使用TextCNN_pth2onnx.py导出onnx文件：

In [14]:
import sys
!{sys.executable} TextCNN_pth2onnx.py --weight_path ./TextCNN_9045_seed460473.pth --onnx_path ./dy_textcnn.onnx

path /opt/huawei/edu-apaas/src/init/textcnn/ascend-textcnn/Chinese-Text-Classification-Pytorch/THUCNews/data/class.txt
Exported graph: graph(%sentence : Long(*, 32, strides=[32, 1], requires_grad=0, device=cpu),
      %embedding.weight : Float(4762, 300, strides=[300, 1], requires_grad=1, device=cpu),
      %convs0.0.weight : Float(256, 1, 2, 30, strides=[60, 60, 30, 1], requires_grad=1, device=cpu),
      %convs0.0.bias : Float(256, strides=[1], requires_grad=1, device=cpu),
      %convs0.1.weight : Float(256, 1, 3, 30, strides=[90, 90, 30, 1], requires_grad=1, device=cpu),
      %convs0.1.bias : Float(256, strides=[1], requires_grad=1, device=cpu),
      %convs0.2.weight : Float(256, 1, 4, 30, strides=[120, 120, 30, 1], requires_grad=1, device=cpu),
      %convs0.2.bias : Float(256, strides=[1], requires_grad=1, device=cpu),
      %convs1.0.weight : Float(256, 256, 1, 10, strides=[2560, 10, 10, 1], requires_grad=1, device=cpu),
      %convs1.0.bias : Float(256, strides=[1], requires_

运行结束上述命令后，可获得dy_textcnn.onnx文件

参数说明：

- --weight_path：权重路径。
- --onnx_path：生成onnx模型的路径。

简化onnx文件：

In [15]:
%cd ..
!mkdir ./onnx_sim_dir
import sys
!{sys.executable} -m onnxsim --overwrite-input-shape="sentence:64,32" ./ascend-textcnn/dy_textcnn.onnx ./onnx_sim_dir/textcnn_64bs_sim.onnx

/opt/huawei/edu-apaas/src/init/textcnn
Simplifying...
Finish! Here is the difference:
┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃            ┃ Original Model ┃ Simplified Model ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ Concat     │ 1              │ 1                │
│ Constant   │ 11             │ 11               │
│ Conv       │ 6              │ 6                │
│ Gather     │ 1              │ 1                │
│ Gemm       │ 1              │ 1                │
│ MaxPool    │ 3              │ 3                │
│ Relu       │ 3              │ 3                │
│ Squeeze    │ 6              │ 6                │
│ Unsqueeze  │ 1              │ 1                │
│ Model Size │ 8.3MiB         │ 8.3MiB           │
└────────────┴────────────────┴──────────────────┘


##### 1.4.2 使用ATC工具将ONNX模型转OM模型

1. 执行命令查看芯片型号

In [16]:
!npu-smi info

+------------------------------------------------------------------------------------------------+
| npu-smi 24.1.rc3                 Version: 24.1.rc3                                             |
+---------------------------+---------------+----------------------------------------------------+
| NPU   Name                | Health        | Power(W)    Temp(C)           Hugepages-Usage(page)|
| Chip                      | Bus-Id        | AICore(%)   Memory-Usage(MB)  HBM-Usage(MB)        |
+===========================+===============+====================================================+
| 2     910B3               | OK            | 91.1        51                0    / 0             |
| 0                         | 0000:C2:00.0  | 0           0    / 0          3382 / 65536         |
+===========================+===============+====================================================+
+---------------------------+---------------+----------------------------------------------------+
| NPU     

2. 执行ATC命令

请根据查看到的芯片型号，修改`soc_version`的值，格式为Ascendxxx，例如Ascend910B4，Ascend910B3

In [18]:
!mkdir ./mg_onnx_dir && mkdir ./mg_om_dir
!atc --model=onnx_sim_dir/textcnn_64bs_sim.onnx --framework=5 --output=mg_om_dir/textcnn_64bs_mg --output_type=FP16 --soc_version=Ascend910B3 --enable_small_channel=1

mkdir: cannot create directory './mg_onnx_dir': File exists
ATC start working now, please wait for a moment.
-\|/...
ATC run success, welcome to the next use.



参数说明：

- model：为ONNX模型文件。

- output：输出的OM模型。

- soc_version：处理器型号。

- log：日志级别。

- output_type：输出数据类型

##### 1.4.3 执行推理

In [19]:
!mkdir ./output_data
import sys
!{sys.executable} -m ais_bench --model mg_om_dir/textcnn_64bs_mg.om --input ./ascend-textcnn/bin --output ./output_data1 --output_dirname bs64 --device 0  

[INFO] acl init success
[INFO] open device 0 success
[INFO] create new context
[INFO] load model mg_om_dir/textcnn_64bs_mg.om success
[INFO] create model description success
[INFO] try get model batchsize:64
[INFO] output path:./output_data1/bs64
[INFO] get filesperbatch files0 size:256 tensor0size:16384 filesperbatch:64 runcount:157
[INFO] warm up 1 done
Inference array Processing: 100%|█████████████| 157/157 [00:05<00:00, 28.28it/s]
[INFO] -----------------Performance Summary------------------
[INFO] NPU_compute_time (ms): min = 0.6376953125, max = 0.75927734375, mean = 0.6708486763535032, median = 0.669921875, percentile(99%) = 0.7337109374999999
[INFO] throughput 1000*batchsize.mean(64)/NPU_compute_time.mean(0.6708486763535032): 95401.54472373924
[INFO] ------------------------------------------------------
[INFO] unload model success, model Id is 1
[INFO] end to reset device 0
[INFO] end to finalize acl


参数说明：

- model：om文件路径。

- input：数据预处理后保存文件的路径。

- output：输出文件夹路径。

- --output_dirname：输出文件夹名。

- device：NPU的ID，默认填0。

推理输出结果在output_data1生成bs64文件夹。

#### 1.5 精度验证
调用TextCNN_postprocess.py脚本，可以获得精度accuracy数据，输入指令后请稍等片刻

In [20]:
%cd ascend-textcnn
import sys
!{sys.executable} TextCNN_postprocess.py ../output_data1/bs64 > result_bs1.json

/opt/huawei/edu-apaas/src/init/textcnn/ascend-textcnn


参数说明：

- `../output_data1/bs64`：为生成推理结果所在相对路径。
- `result_bs1.json`：为保存精度数值的文件。文件路径位于ascend-textcnn下

result_bs1.json内容：

TOP1:  0.9047

TOP5:  0.9935

#### 1.6 性能验证
可使用ais_bench推理工具的纯推理模式验证不同batch_size的om模型的性能，参考命令如下：

In [21]:
%cd ..
import sys
!{sys.executable} -m ais_bench --model mg_om_dir/textcnn_64bs_mg.om

/opt/huawei/edu-apaas/src/init/textcnn
[INFO] acl init success
[INFO] open device 0 success
[INFO] create new context
[INFO] load model mg_om_dir/textcnn_64bs_mg.om success
[INFO] create model description success
[INFO] try get model batchsize:64
[INFO] warm up 1 done
Inference array Processing: 100%|████████████████| 1/1 [00:00<00:00, 450.56it/s]
[INFO] -----------------Performance Summary------------------
[INFO] NPU_compute_time (ms): min = 0.7399997711181641, max = 0.7399997711181641, mean = 0.7399997711181641, median = 0.7399997711181641, percentile(99%) = 0.7399997711181641
[INFO] throughput 1000*batchsize.mean(64)/NPU_compute_time.mean(0.7399997711181641): 86486.51323674586
[INFO] ------------------------------------------------------
[INFO] unload model success, model Id is 1
[INFO] end to reset device 0
[INFO] end to finalize acl


参数说明：
- model：om模型的路径

从上述结果可以看到类似如下得性能统计数据（不同设备，不同条件下数据不同，下面内容只做参考）：
```
NPU_compute_time (ms): min = 0.7960004806518555, max = 0.7960004806518555, mean = 0.7960004806518555, median = 0.7960004806518555, percentile(99%) = 0.7960004806518555
```

#### 1.7 模型推理性能&精度
调用ACL接口推理计算，性能参考下列数据，官网精度为91.22。

|芯片型号	|Batch Size	|数据集	|精度TOP1	|精度TOP5	|性能|
| :----- | :----- | :----- | :----- | :----- | :----- |
|300I Pro	|1	|ImageNet	|90.47	|99.35	|5140.8|
|300I Pro	|4	|ImageNet	|90.47	|99.35	|16167.36|
|300I Pro	|8	|ImageNet	|90.47	|99.35	|21826.91|
|300I Pro	|16	|ImageNet	|90.47	|99.35	|25861.78|
|300I Pro	|32	|ImageNet	|90.47	|99.35	|27918.85|
|300I Pro	|64	|ImageNet	|90.47	|99.35	|29237.18|